In [ ]:
# Importing required packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers


import numpy as np
import pandas as pd
import os
import shutil
import random
tf.__version__

In [ ]:
import glob
# Path of the data
#train_path = 'covid19_data/train'
Patient_list = '/.../Covid19_detection/Patients/'
no_patients = 20

train_path_list = []
for i in range(no_patients):
    train_patients = glob.glob(Patient_list + 'Patient_' + str(i))
    train_path_list.append(train_patients[0])
print(train_path_list)
valid_path = 'covid19_data/val'
test_path = 'covid19_data/test'
#Randomly select Patients for Training and Validation 80/20 Split

In [ ]:
# Mobile Net(v1)
for i in range(no_patients):
    train_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path_list[i], target_size=(224, 224), classes=['negative', 'positive'], batch_size=2)
    print(train_batches)
valid_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224, 224), classes=['negative', 'positive'],  batch_size=1)
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path, target_size=(224, 224), classes=['negative', 'positive'], batch_size=1, shuffle=False)

In [ ]:
from tensorflow.keras import Model

Model_array = []
for i in range(no_patients):
    mobile = tf.keras.applications.mobilenet.MobileNet()
    mobile.summary()
    x = mobile.layers[-6].output
    output = Dense(units=2, activation='softmax')(x)
    model_mobile = Model(inputs=mobile.input, outputs=output)
    model_mobile.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    Model_array.append(model_mobile)
    print(Model_array)


In [ ]:
for layer in mobile.layers[:-23]:
    layer.trainable=False
max_rounds = 10

# def get_model():
#     inputs = keras.Input(shape=(128,))
#     outputs = layers.Dense(1)(inputs)
#     return keras.Model(inputs, outputs)

#Global_model = None
no_patients = 20

for rounds in range(max_rounds):
    for i in range(int(no_patients)):
        Model_array[i].fit(x=train_batches, validation_data=valid_batches, epochs=1, verbose=1)

    weights = [model.get_weights() for model in Model_array]
    new_weights = []
    wlen = len(weights[0])
    for i in range(wlen):
        w_mean = 0
        for j in range(no_patients):
            w_mean = w_mean + weights[j][i]
        new_weights = new_weights + [w_mean / float(no_patients)]

    [model.set_weights(new_weights) for model in Model_array]
    
#    for model in Model_array:
#        Global_model.append(new_weights)

# for rounds in range(max_rounds):
# #    for i in range(int(no_patients)):
# #        weights = [int(no_patients[i]) + int(no_patients[i+1]) for i in range(no_patients) - 1)]
# #        inputs = keras.Input(shape=(128,))
#         Model_array[i].fit(x=train_batches, validation_data=valid_batches, epochs=1, verbose=1)
# #        outputs = layer.average(Model_array)
# #   inputs = keras.Input(shape=(128,))

# #    outputs = layers.average(Model_array)
# #    Global_model = outputs
# #    averaged_model = keras.Model(inputs=inputs, outputs=outputs)

# #    Global_model = np.average(Model_array)
# #        Global_model = keras.Model(inputs=inputs, outputs=outputs)
# #    for i in range(no_patients):
# #        Model_array = Global_model

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, accuracy_score

sns.set()

def confusion_matrix_and_report(model, batch_to_pred):
  preds = model.predict(x=batch_to_pred)
  y_preds = np.argmax(preds, axis=-1)
  y_true = batch_to_pred.classes
  accuracy = accuracy_score(y_true, y_preds)
  print(f"The accuracy of the model is: {accuracy}")
  confuse = confusion_matrix(y_true, y_preds)
  report = classification_report(y_true, y_preds)
  print(report)    
  sns.heatmap(pd.DataFrame(confuse), annot=True, cmap='viridis', fmt='g')
  plt.tight_layout()
  plt.xlabel('Predicted label')
  plt.ylabel('Actual label')
  plt.title('CONFUSION MATRIX')
  plt.show()

def roc_auc(model, batch_to_pred):
  proba_list = []
  preds = model.predict(x=batch_to_pred)[::,1]
  y_true = batch_to_pred.classes
  fpr, tpr, thresholds = roc_curve(y_true, preds)
  auc = roc_auc_score(y_true, preds)
  plt.plot(fpr,tpr,label="Auc="+str(auc))
  plt.legend(loc=4)

In [ ]:
confusion_matrix_and_report(model_mobile, test_batches)
roc_auc(model_mobile, test_batches)

In [ ]:
model_mobile.save('model_mobilenet.h5')
print("SAVED")

In [ ]:
# Predicting our own images

import numpy as np
from tensorflow.keras.models import load_model
from keras.preprocessing import image

model_dir = 'model_mobilenet.h5'
image_dir = 'covid19_data/test/negative/NORMAL2-IM-0381-0001.jpeg'

model_loaded = load_model(model_dir)
img_width, img_height = 224, 224
img = image.load_img(image_dir, target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

result = np.argmax(model_loaded.predict(img), axis=-1)[0]
if(result==1):
  print("COVID19 POSITIVE! TAKE CARE")
else:
  print("COVID19 NEGATIVE! STAY SAFE")